#Google Colab GPU 

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
import os
os.chdir('/gdrive/My Drive/Huawei')

#Kütüphaneler

In [ ]:
import numpy as np
import pandas as pd
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Veri çekme


In [ ]:
root = '/gdrive/My Drive/Huawei/'
data = pd.read_csv(root + 'film_data.csv')
print(data)

       Unnamed: 0 date_published  ... avg_vote  votes
0               0     1972-10-28  ...      7.2  11023
1               1     2011-03-26  ...      3.8    145
2               2     1949-02-02  ...      6.6    174
3               3     2009-05-31  ...      5.2    108
4               4     1962-11-01  ...      5.1    127
...           ...            ...  ...      ...    ...
70685       70685     1988-04-14  ...      6.5    168
70686       70686     2014-04-18  ...      6.9  23046
70687       70687     2013-02-14  ...      7.2    209
70688       70688     2016-01-14  ...      7.1    863
70689       70689     1939-09-04  ...      7.5   5376

[70690 rows x 9 columns]


#VERİ ÖNİŞLEME

##Verinin atanması ve Metin temizleme kümeleri

In [ ]:
description = data["description"]
nok_isaretleri_kümesi = string.punctuation
etkisiz_kelimeler_kümesi = ["year","film","0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"]


## Veri Temizleme

In [ ]:
def veri_temizleme(metin):
    metin = metin.lower()
    metin = metin.replace("\\n", " ")# Verisetimizdeki yeni satır karekterleri, boşluk karekteriyle değiştirdik.
    # Kesme işareti ve sonrasındaki karekterlerin kaldırılması
    metin = re.sub("’(\w+)", "", metin)
    metin = re.sub("'(\w+)", "", metin)
    metin = re.sub("[“,‘,’,”]", "", metin)
    # Sayıların Kaldırılması
    metin = re.sub("[0-9]+", "", metin)
    # Noktalama işaretlerinin kaldırılması
    metin = "".join(list(map(lambda x: x if x not in nok_isaretleri_kümesi else " ", metin)))
    # Etkisiz kelimelerin bir kısmının kaldırılması
    metin = " ".join([i for i in metin.split() if i not in etkisiz_kelimeler_kümesi])
    # Metinde tek kalan harfleri de çıkartalım
    metin = " ".join([i for i in metin.split() if len(i) > 1])

    return metin

## Veri Temizleme İşlemi

In [ ]:
temiz_data = description.apply(lambda x:veri_temizleme(x))

## Veri Lemmatizer

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
lemmatized_data=[]
for i in temiz_data:
  word_list = nltk.word_tokenize(i)
  lemmatized_data.append(' '.join([lemmatizer.lemmatize(w) for w in word_list]))
lemmatized_data = pd.Series(lemmatized_data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Veri Tokenizer

In [ ]:
tokenized_data = lemmatized_data.apply(lambda x: x.split())
print(tokenized_data)

0        [men, drag, racing, chevy, dennis, wilson, mec...
1        [christmas, weekend, quaint, texas, hotel, joe...
2        [girl, impoverished, family, jilted, rich, fia...
3        [jack, newman, lecture, local, university, pre...
4        [guitar, playing, traveler, murder, charge, jo...
                               ...                        
70685    [illegal, immigrant, canada, survive, kickboxi...
70686    [movie, chronicle, chetan, met, wife, difficul...
70687    [rectoverso, omnibus, anthology, indonesian, m...
70688    [handing, girlfriend, favorite, dog, dog, catc...
70689    [timid, british, army, officer, quit, burn, da...
Length: 70690, dtype: object


## Kelime Listesi

In [ ]:
kelime_listesi = gensim.corpora.Dictionary(tokenized_data)

print(kelime_listesi)


Dictionary(56760 unique tokens: ['chevy', 'dennis', 'drag', 'driver', 'james']...)


## Döküman Terim Matrisi


In [ ]:
dokuman_terim_matrisi = [kelime_listesi.doc2bow(terim) for terim in tokenized_data]


#LDA Model

In [ ]:
from gensim.models import CoherenceModel
konu_sayisi_aralik_listesi = range(9,25,1)
print(konu_sayisi_aralik_listesi)

range(9, 25)


## LDA Model Parametre Belirlenmesi

In [ ]:
tutarlilik_skorlar_listesi = []
konu_sayisi_listesi = []

for konu_sayisi in konu_sayisi_aralik_listesi:
    lda_model = gensim.models.ldamodel.LdaModel(corpus = dokuman_terim_matrisi,
                                               id2word = kelime_listesi,
                                               num_topics = konu_sayisi, 
                                               passes = 15)
    
    tutarlilik_model_lda = CoherenceModel(model=lda_model, texts=tokenized_data, 
                                          dictionary=kelime_listesi, coherence='c_v')
    gecici_tutarlilik_skoru_lda = tutarlilik_model_lda.get_coherence()
    tutarlilik_skorlar_listesi.append(gecici_tutarlilik_skoru_lda)
    konu_sayisi_listesi.append(konu_sayisi)
    print(tutarlilik_skorlar_listesi)
    print(konu_sayisi_listesi)

[0.37266655998688064]
[9]
[0.37266655998688064, 0.33963229390398014]
[9, 10]
[0.37266655998688064, 0.33963229390398014, 0.35927361075903647]
[9, 10, 11]
[0.37266655998688064, 0.33963229390398014, 0.35927361075903647, 0.3630872553451691]
[9, 10, 11, 12]
[0.37266655998688064, 0.33963229390398014, 0.35927361075903647, 0.3630872553451691, 0.36979973716298875]
[9, 10, 11, 12, 13]
[0.37266655998688064, 0.33963229390398014, 0.35927361075903647, 0.3630872553451691, 0.36979973716298875, 0.3276699351477612]
[9, 10, 11, 12, 13, 14]
[0.37266655998688064, 0.33963229390398014, 0.35927361075903647, 0.3630872553451691, 0.36979973716298875, 0.3276699351477612, 0.359275719549594]
[9, 10, 11, 12, 13, 14, 15]
[0.37266655998688064, 0.33963229390398014, 0.35927361075903647, 0.3630872553451691, 0.36979973716298875, 0.3276699351477612, 0.359275719549594, 0.3569468386187295]
[9, 10, 11, 12, 13, 14, 15, 16]
[0.37266655998688064, 0.33963229390398014, 0.35927361075903647, 0.3630872553451691, 0.36979973716298875, 

In [ ]:
from gensim.models import CoherenceModel
konu_sayisi_aralik_listesi = range(5,10,1)

tutarlilik_skorlar_listesi = []
konu_sayisi_listesi = []

for konu_sayisi in konu_sayisi_aralik_listesi:
    lda_model = gensim.models.ldamodel.LdaModel(corpus = dokuman_terim_matrisi,
                                               id2word = kelime_listesi,
                                               num_topics = 9, 
                                               passes = 15,
                                               eval_every=1 )
    
    tutarlilik_model_lda = CoherenceModel(model=lda_model, texts=tokenized_data, 
                                          dictionary=kelime_listesi, coherence='c_v')
    gecici_tutarlilik_skoru_lda = tutarlilik_model_lda.get_coherence()
    tutarlilik_skorlar_listesi.append(gecici_tutarlilik_skoru_lda)
    konu_sayisi_listesi.append(konu_sayisi)
    print(tutarlilik_skorlar_listesi)
    print(konu_sayisi_listesi)

## Modelin Parametreleri Aşağıdaki Gibi Seçilmiştir

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = dokuman_terim_matrisi,
                                           id2word = kelime_listesi,
                                           num_topics = 9,
                                           passes = 15)
lda_model.save('lda_train.model3')

#LDA Modelin Topikleri

In [ ]:
konular = lda_model.print_topics(num_words=7) 

for konu in konular:
    print(konu)

(0, '0.031*"school" + 0.022*"student" + 0.019*"high" + 0.012*"college" + 0.011*"friend" + 0.007*"bank" + 0.007*"teacher"')
(1, '0.016*"war" + 0.013*"story" + 0.010*"love" + 0.007*"singer" + 0.007*"star" + 0.007*"band" + 0.005*"set"')
(2, '0.022*"town" + 0.016*"small" + 0.013*"murder" + 0.012*"police" + 0.008*"local" + 0.008*"detective" + 0.007*"crime"')
(3, '0.012*"killer" + 0.010*"group" + 0.009*"war" + 0.006*"man" + 0.006*"agent" + 0.006*"revenge" + 0.006*"gang"')
(4, '0.032*"love" + 0.028*"woman" + 0.025*"young" + 0.019*"man" + 0.018*"life" + 0.018*"fall" + 0.012*"girl"')
(5, '0.031*"story" + 0.017*"movie" + 0.016*"life" + 0.015*"comedy" + 0.014*"based" + 0.009*"director" + 0.009*"set"')
(6, '0.029*"life" + 0.019*"young" + 0.015*"family" + 0.015*"friend" + 0.012*"man" + 0.011*"year" + 0.011*"woman"')
(7, '0.006*"human" + 0.005*"ancient" + 0.005*"treasure" + 0.005*"destroy" + 0.004*"century" + 0.004*"power" + 0.004*"air"')
(8, '0.007*"american" + 0.006*"earth" + 0.006*"spy" + 0.006*"

# En çok uyuşan topiği alma

In [ ]:
class_names = ["0","1","2","3","4","5","6","7","8"]
son_sonuc = []
for j in range(len(temiz_data)):
  bow = kelime_listesi.doc2bow(tokenized_data[j])
  results = lda_model.get_document_topics(bow, minimum_probability= 0.0, per_word_topics=False)
  sonuc = []
  for i in range(9):
    sonuc.append(results[i][1])
  son_sonuc.append(class_names[np.argmax([sonuc])])

In [ ]:
son_sonuc = pd.DataFrame(son_sonuc)
print(son_sonuc)

       0
0      6
1      4
2      4
3      6
4      4
...   ..
70685  2
70686  6
70687  8
70688  7
70689  3

[70690 rows x 1 columns]


# Elde edilen sonucu csv ye çevirme

In [ ]:
compression_opts = dict(method='zip',
                        archive_name='out.csv')  
son_sonuc.to_csv('out.zip', index=False,
          compression=compression_opts)  